# classiq

I will start with small 2 qubit problems, one solvable and one not.

In [1]:
solvable_problem = [1,1] #An solvable instance of the partitioning problem to be solved using 2 quibts
unsolvable_problem = [1,3] #An unsolvable instance of the partitioning problem to be solved using 2 quibts

In [2]:
%pip install qiskit
%pip install classiq
%pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 5.1 MB/s 
     |████████████████████████████████| 19.2 MB 66.6 MB/s 
     |████████████████████████████████| 240 kB 8.1 MB/s 
     |████████████████████████████████| 55 kB 1.3 MB/s 
     |████████████████████████████████| 106 kB 46.1 MB/s 
     |████████████████████████████████| 49 kB 2.9 MB/s 
     |████████████████████████████████| 943 kB 43.1 MB/s 
     |████████████████████████████████| 50 kB 2.1 MB/s 
     |████████████████████████████████| 37.5 MB 1.6 MB/s 
     |████████████████████████████████| 4.1 MB 31.0 MB/s 
     |████████████████████████████████| 1.9 MB 41.7 MB/s 
     |████████████████████████████████| 112 kB 41.0 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.2-py3-none-any.whl size=12224 sha256=2eadb83f90abe5835bdb294be9f12b0541e8448b1e7ea9eccf3037427deed4ee
  Stored in directory: /root/.cache/pip/wheels/30/dc/90/0ee55

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nest_asyncio
nest_asyncio.apply()
import classiq

classiq.authenticate()

Your user code: CKWQ-MQXW
If a browser doesn't automatically open, please visit the url: https://classiq.eu.auth0.com/activate?user_code=CKWQ-MQXW


In [4]:
from classiq import ModelDesigner, Executor
from classiq.builtin_functions import UCC, HartreeFock

from classiq.interface.chemistry.molecule import Molecule
from classiq.interface.chemistry.ground_state_problem import GroundStateProblem
from classiq.interface.backend.backend_preferences import (
    IBMBackendPreferences,
    IBMBackendProvider,
)
from classiq.interface.executor.execution_preferences import ExecutionPreferences, OptimizerPreferences
from classiq.interface.executor.hamiltonian_minimization_problem import HamiltonianMinimizationProblem
from classiq.interface.executor.quantum_instruction_set import QuantumInstructionSet
from classiq.interface.executor.quantum_program import QuantumProgram
from classiq.interface.chemistry.operator import PauliOperator
from classiq import ModelDesigner
from classiq.builtin_functions import HardwareEfficientAnsatz

## preparing the hamiltonian for 2 quibts (Ising model)

In [5]:
def create_hamiltonian(S):
  n_1 = S[0]
  n_2 = S[1]

  H = [("II",n_1*n_1),
 ("ZZ",n_1*n_2),
 ("ZZ",n_2*n_1),
 ("II",n_2*n_2)]
  return H


## Prepare simulator

In [6]:
def define_simulator(simulator_name):
    if(simulator_name == "qasm" or simulator_name == "QASM"):
      curr_simulator = "qasm_simulator"
    else:
      curr_simulator = "aer_simulator_statevector"
    return curr_simulator

In [7]:
def define_optimizer(optimizer_name):
    if(optimizer_name == "cobyla" or optimizer_name == "COBYLA"):
      curr_optimizer = "COBYLA"
    else:
      curr_optimizer = "L_BFGS_B"
    return curr_optimizer

In [8]:
def prepare_simulator_classiq(curr_simulator, curr_optimizer,num_of_qubits=2, anzatz_first_gate = "x"):
    NUM_QUBITS = num_of_qubits
    FULLY_CONNECTED_MESH = [[0, 1]]
    hwea_params = HardwareEfficientAnsatz(
        num_qubits=NUM_QUBITS,
        connectivity_map=FULLY_CONNECTED_MESH,
        one_qubit_gates=[anzatz_first_gate, "ry"],
        two_qubit_gates="cx",
        reps=3,
    )

    model_designer = ModelDesigner()
    model_designer.HardwareEfficientAnsatz(params=hwea_params)
    circuit = model_designer.synthesize()
    #circuit.show()
    circuit_qasm = circuit.qasm
    optimizer_preferences = OptimizerPreferences(
        max_iteration=60,
        name = curr_optimizer
    )

    ibm_provider = IBMBackendProvider(
        #insert your information
    )

    backend_preferences = IBMBackendPreferences(
        backend_name=curr_simulator,
        access_token = "insert your information" ,
        provider=ibm_provider,
    )

    num_shots = 8192

    execution_preferences = ExecutionPreferences(
        num_shots=num_shots,
        backend_preferences=backend_preferences,
        optimizer_preferences=optimizer_preferences
    )
    return execution_preferences, circuit_qasm

## compute minimal eigenvalue

In [19]:
def minimum_eigenvalue_by_vqe(H, circuit_qasm, execution_preferences):
  hamiltonian_problem = HamiltonianMinimizationProblem(
      ansatz=QuantumProgram(code=circuit_qasm, syntax=QuantumInstructionSet.QASM),
      hamiltonian=H
  )
  result = Executor(
      preferences=execution_preferences
  ).execute_hamiltonian_minimization(hamiltonian_problem)
  print("minimal eigenvalue: ", format(result.energy,".3f"))

## create problem instance

In [20]:
def calc_operator(problem):
  partitioningProblem = create_hamiltonian(problem)
  operator= PauliOperator(pauli_list=partitioningProblem)
  print(operator)
  return operator

In [21]:
default_operator = calc_operator(solvable_problem) 

version='0.18.2' pauli_list=[('II', (1+0j)), ('ZZ', (1+0j)), ('ZZ', (1+0j)), ('II', (1+0j))]


## run experiment

In [22]:
def run_experiment(simulator,optimizer,num_quibits=2,anzatz_first_gate = "x",operator= default_operator):
  curr_simulator = define_simulator(simulator)
  curr_optimizer = define_optimizer(optimizer)
  execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer, num_quibits, anzatz_first_gate)
  minimum_eigenvalue_by_vqe(operator, circuit_qasm, execution_preferences)


In [31]:
def test_problem_instance(operator, num_quibits=2):
  #run_experiment("qasm","cobyla",num_quibits,"x",operator)   //returns error                          
  #run_experiment("qasm","bfgs",num_quibits,"x",operator)    //returns error 
  print(f"simulator: aer_statevector, optimizer: cobyla, num of quibits: {num_quibits}, anzats: x, ", end = ' ')                            
  r1 = run_experiment("aer_statevector","cobyla",num_quibits,"x",operator)  
  print(f"simulator: aer_statevector, optimizer: cobyla, num of quibits: {num_quibits}, anzats: RyRz, ", end = ' ')                
  r2 = run_experiment("aer_statevector","cobyla",num_quibits,"rz",operator) 
  print(f"simulator: aer_statevector, optimizer: bfgs, num of quibits: {num_quibits}, anzats: x, ", end = ' ')          
  r3 = run_experiment("aer_statevector","bfgs",num_quibits,"x",operator) 
  print(f"simulator: aer_statevector, optimizer: bfgs, num of quibits: {num_quibits}, anzats: RyRz, ", end = ' ')                   
  r4 = run_experiment("aer_statevector","bfgs",num_quibits,"rz",operator)  
  return r1, r2, r3, r4           

# Run experiments

In [32]:
print("Solving the solvable problem ", end = '')
print(solvable_problem)
solvable_operator = calc_operator(solvable_problem)
r1_1, r2_1, r3_1, r4_1 = test_problem_instance(solvable_operator)

Solving the solvable problem [1, 1]
version='0.18.2' pauli_list=[('II', (1+0j)), ('ZZ', (1+0j)), ('ZZ', (1+0j)), ('II', (1+0j))]
simulator: aer_statevector, optimizer: cobyla, num of quibits: 2, anzats: x,  minimal eigenvalue:  0.000
simulator: aer_statevector, optimizer: cobyla, num of quibits: 2, anzats: RyRz,  minimal eigenvalue:  0.004
simulator: aer_statevector, optimizer: bfgs, num of quibits: 2, anzats: x,  minimal eigenvalue:  0.000
simulator: aer_statevector, optimizer: bfgs, num of quibits: 2, anzats: RyRz,  minimal eigenvalue:  0.001


In [33]:
print("Solving the unsolvable problem ", end = '')
print(unsolvable_problem)
unsolvable_operator = calc_operator(unsolvable_problem)
r1_2, r2_2, r3_2, r4_2 = test_problem_instance(unsolvable_operator)

Solving the unsolvable problem [1, 3]
version='0.18.2' pauli_list=[('II', (1+0j)), ('ZZ', (3+0j)), ('ZZ', (3+0j)), ('II', (9+0j))]
simulator: aer_statevector, optimizer: cobyla, num of quibits: 2, anzats: x,  minimal eigenvalue:  4.000
simulator: aer_statevector, optimizer: cobyla, num of quibits: 2, anzats: RyRz,  minimal eigenvalue:  4.002
simulator: aer_statevector, optimizer: bfgs, num of quibits: 2, anzats: x,  minimal eigenvalue:  4.001
simulator: aer_statevector, optimizer: bfgs, num of quibits: 2, anzats: RyRz,  minimal eigenvalue:  4.004


# Results for 2 qubits problems instances

## Solvable problem ([1,1])

* The qasm simulator **didn't work at all** (got stuck in infinite loop or threw error).

* The aer_statevector simulator worked well with the x anzats but gave wrong results with the RyRz anzats.








Optimizer/Anzats  | x               | RyRz
------------------|:----------------|--------------------
Cobyla            |  0.0 (worked!)  |  0.004 (wrong result)
bfgs              |  0.0 (worked!)  |  0.001 (wrong result)

## Unsolvable problem ([1,3])

* The qasm simulator **didn't work at all** (got stuck in infinite loop or threw error).

* The aer_statevector simulator worked well **only with cobyla optimizer and x anzats**, and gave wrong results for all the other options.








Optimizer/Anzats  | x               | RyRz
------------------|:----------------|--------------------
Cobyla            |  4.0 (worked!)  |  4.002 (wrong result)
bfgs              |  4.001 (wrong result)| 4.004 (wrong result)

## General Form

In [35]:
solvable_problem_3_qubits = [1,2,1]
unsolvable_problem_3_qubits = [1,3,1]

### preparing the hamiltonian for n quibts (Ising model)

In [34]:
import numpy as np
from qiskit.opflow import Z, I

def multiply_ising_models(ham1, ham2, coef1, coef2):
  final_sigma = ""
  final_coef = 0
  for i in range(len(ham1)):
      sigma1 = ham1[i]
      sigma2 = ham2[i]
      if (sigma1 == 'Z' and sigma2 == 'Z'):
        final_sigma += 'I'
      elif (sigma1 == 'I' and sigma2 == 'Z'):
        final_sigma += 'Z'
      elif (sigma1 == 'Z' and sigma2 == 'I'):
        final_sigma += 'Z'
      else:
        final_sigma += 'I'
  return (final_sigma,coef1*coef2)


def create_general_hamiltonian_classiq(S):
  num_qubits = len(S)
  operator = []
  sigmas_list = []
  for qubit in range(num_qubits):
    if qubit == 0:
       sigma = "Z"
    else:
       sigma = "I"
    for i in range(1,num_qubits):
      if i == qubit:
        sigma = sigma + "Z"
      else:
        sigma = sigma + "I"
    sigmas_list.append((sigma,S[qubit]))

  for i in range(len(sigmas_list)):
    for j in range(len(sigmas_list)):
      sigma = multiply_ising_models(sigmas_list[i][0], sigmas_list[j][0], sigmas_list[i][1], sigmas_list[j][1])
      operator.append(sigma)

  H = operator
  final_operator= PauliOperator(pauli_list=H)
  return final_operator 


# Run experiments

## 3 quibit problems

In [36]:
print("Solving the solvable problem ", end = '')
print(solvable_problem_3_qubits)
solvable_operator = create_general_hamiltonian_classiq(solvable_problem_3_qubits)
test_problem_instance(solvable_operator, 3)

Solving the solvable problem [1, 2, 1]
simulator: aer_statevector, optimizer: cobyla, num of quibits: 3, anzats: x,  minimal eigenvalue:  0.004
simulator: aer_statevector, optimizer: cobyla, num of quibits: 3, anzats: RyRz,  minimal eigenvalue:  0.213
simulator: aer_statevector, optimizer: bfgs, num of quibits: 3, anzats: x,  minimal eigenvalue:  0.004
simulator: aer_statevector, optimizer: bfgs, num of quibits: 3, anzats: RyRz,  minimal eigenvalue:  0.006


(None, None, None, None)

In [38]:
print("Solving the unsolvable problem ", end = '')
print(unsolvable_problem_3_qubits)
unsolvable_operator = create_general_hamiltonian_classiq(unsolvable_problem_3_qubits)
test_problem_instance(unsolvable_operator, 3)

Solving the unsolvable problem [1, 3, 1]
simulator: aer_statevector, optimizer: cobyla, num of quibits: 3, anzats: x,  minimal eigenvalue:  1.001
simulator: aer_statevector, optimizer: cobyla, num of quibits: 3, anzats: RyRz,  minimal eigenvalue:  1.555
simulator: aer_statevector, optimizer: bfgs, num of quibits: 3, anzats: x,  minimal eigenvalue:  1.029
simulator: aer_statevector, optimizer: bfgs, num of quibits: 3, anzats: RyRz,  minimal eigenvalue:  1.018


(None, None, None, None)

# Results for 3 qubits problems instances

* The qasm simulator **didn't work at all** (got stuck in infinite loop or threw error).

* The aer_statevector unfourtunatly returned wrong result for all the options, especially for the cobyla-RyRz anzats combination.

## Solvable problem ([1,2,1])








Optimizer/Anzats  | x               | RyRz
------------------|:----------------|--------------------
Cobyla            |  0.004 (wrong result)  |  0.213 (very wrong result)
bfgs              |  0.004 (wrong result)| 0.006 (wrong result)

## Unsolvable problem ([1,3,1])












Optimizer/Anzats  | x               | RyRz
------------------|:----------------|--------------------
Cobyla            |  1.001 (wrong result)  | 1.555 (very wrong result)
bfgs              |  1.029 (wrong result)| 1.018 (wrong result)